In [29]:
import os
os.chdir(r"D:\108-1\database\project\database_design")

In [30]:
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile

In [31]:
import codecs
file = codecs.open("..\sql_command\insertCommand.sql", 'w', "utf-8")

In [32]:
def commentWriter(file, comment):
    file.write('\n')
    for i in comment.split('\n'):
        file.write("-- " + i + '\n')
    file.write('\n')

In [33]:
file.write("USE delivery_db;\n")

In [34]:
# deal with User table-------------------------------------------------------
############### write commment starts ###############
commentWriter(file, "deal with User table")
################ write commment ends ################
User = pd.read_excel('database_table.xlsx', sheet_name='User')
userQuery  = "INSERT INTO User (UName, account, password, address, phone) values\n"
userTalbeSize = len(User.index)
count = 0
for row in User.itertuples():
    userQuery += '("{}", "{}", "{}", "{}", "{}")'.format(getattr(row, "UName"),
                                                        getattr(row, "account"), 
                                                        getattr(row, "password"),
                                                        getattr(row, "address"),
                                                        '0' + str(getattr(row, "phone")))
    if(count < userTalbeSize - 1):
        userQuery += ',\n'
    else:
        userQuery += ';'
    count += 1
file.write(userQuery + '\n')

In [35]:
import numpy as np
def transferURL(oriURL):
    if(pd.isna(oriURL)):
        return "NULL"
    return r"\\".join(oriURL.split('\\'))

In [36]:
def formatGet(row, columnName):
    data = getattr(row, columnName)
    if(pd.isna(data)):
        return "NULL"
    else:
        if(isinstance(data, str) or 
           isinstance(data, pd._libs.tslibs.timestamps.Timestamp)):
            return '"{}"'.format(data)
        else:
            return data

In [37]:
# deal with Platform table-------------------------------------------------------
############### write commment starts ###############
commentWriter(file, "deal with Platform table")
################ write commment ends ################

platform = pd.read_excel('database_table.xlsx', sheet_name='Platform')
platformQuery  = "INSERT INTO Platform (PName, shippingFee, P_IMG_URL) values\n"
platformTalbeSize = len(platform.index)
count = 0
for row in platform.itertuples():
    platformQuery += '("{}", {}, "{}")'.format(getattr(row, "PName"),
                                                getattr(row, "shippingFee"), 
                                                transferURL(getattr(row, "P_IMG_URL")))
    if(count < platformTalbeSize - 1):
        platformQuery += ',\n'
    else:
        platformQuery += ';'
    count += 1
file.write(platformQuery + '\n')
# print(platformQuery)

In [38]:
# deal with Platform_activity table-------------------------------------------------------
############### write commment starts ###############
commentWriter(file, "deal with Platform_activity table")
################ write commment ends ################

PA = pd.read_excel('database_table.xlsx', sheet_name='Platform_activity')
PA_TalbeSize = len(PA.index)
PA_Query = str()
for row in PA.itertuples():
    insertCommand  = "INSERT INTO Platform_activity" + \
                     "(PID, PA_Name, discount_shippingFee, foodDiscountMode, FDM_arg1, FDM_arg2, " + \
                     "description, P_ACT_IMG_URL, start_time, end_time)\n"
    
    sqlCommand = '(SELECT PID, {}, {}, {}, {}, {}, {}, {}, {}, {}\n' + \
                 'FROM Platform\n' + \
                 'WHERE PName = {})'
    
    PA_Query += insertCommand + sqlCommand.format(
        formatGet(row, "PA_Name"), formatGet(row, "discount_shippingFee"), 
        formatGet(row, "foodDiscountMode"), formatGet(row, "FDM_arg1"), 
        formatGet(row, "FDM_arg2"), formatGet(row, "description"), 
        transferURL(formatGet(row, "P_ACT_IMG_URL")),
        formatGet(row, "start_time"), formatGet(row, "end_time"), 
        formatGet(row, "PName"))
    PA_Query += ';\n'

file.write(PA_Query + '\n')
# print(PA_Query)

In [39]:
# deal with Restaurant table-------------------------------------------------------
############### write commment starts ###############
commentWriter(file, "deal with Restaurant table")
################ write commment ends ################

Restaurant = pd.read_excel('database_table.xlsx', sheet_name='Restaurant')
RestaurantQuery  = "INSERT INTO Restaurant (RName, address, R_IMG_URL) values\n"
RestaurantTalbeSize = len(Restaurant.index)
count = 0
for row in Restaurant.itertuples():
    RestaurantQuery += '({}, {}, {})'.format(formatGet(row, "RName"),
                                                formatGet(row, "address"), 
                                                transferURL(formatGet(row, "R_IMG_URL")))
    if(count < RestaurantTalbeSize - 1):
        RestaurantQuery += ',\n'
    else:
        RestaurantQuery += ';'
    count += 1
file.write(RestaurantQuery + '\n')
# print(RestaurantQuery)

In [40]:
# deal with supportedPlatform table-------------------------------------------------------
############### write commment starts ###############
commentWriter(file, "deal with supportedPlatform table")
################ write commment ends ################

SP = pd.read_excel('database_table.xlsx', sheet_name='supportedPlatform')
SP_TalbeSize = len(SP.index)
SP_Query = str()
for row in SP.itertuples():
    insertCommand  = "INSERT INTO supportedPlatform (RID, PID)\n"
    sqlCommand = '(SELECT RID, PID\n' + \
                 'FROM Restaurant, Platform\n' + \
                 'WHERE Restaurant.RName = {} and Platform.PName = {})'
    
    SP_Query += insertCommand + \
                sqlCommand.format(
                    formatGet(row, "RName"), 
                    formatGet(row, "PName"))
    SP_Query += ';\n'

file.write(SP_Query + '\n')
# print(SP_Query)

In [41]:
# deal with RestaurantCategory table-------------------------------------------------------
############### write commment starts ###############
commentWriter(file, "deal with RestaurantCategory table")
################ write commment ends ################

RC = pd.read_excel('database_table.xlsx', sheet_name='RestaurantCategory')
RC_TalbeSize = len(RC.index)
RC_Query = str()
for row in RC.itertuples():
    insertCommand  = "INSERT INTO RestaurantCategory (RID, CName)\n"
    sqlCommand = '(SELECT RID, {}\n' + \
                 'FROM Restaurant\n' + \
                 'WHERE Restaurant.RName = {})'
    RC_Query += insertCommand + \
                sqlCommand.format(
                    formatGet(row, "CName"), 
                    formatGet(row, "RName"))
    RC_Query += ';\n'

file.write(RC_Query + '\n')
# print(RC_Query)

In [42]:
# deal with Food table-------------------------------------------------------
############### write commment starts ###############
commentWriter(file, "deal with Food table")
################ write commment ends ################

Food = pd.read_excel('database_table.xlsx', sheet_name='Food')
Food_TalbeSize = len(Food.index)
Food_Query = str()
for row in Food.itertuples():
    insertCommand  = "INSERT INTO Food (RID, FName, price, F_IMG_URL)\n"
    sqlCommand = '(SELECT RID, {}, {}, {}\n' + \
                 'FROM Restaurant\n' + \
                 'WHERE Restaurant.RName = {})'
    Food_Query += insertCommand + \
                sqlCommand.format(
                    formatGet(row, "FName"),
                    formatGet(row, "price"),
                    transferURL(formatGet(row, "F_IMG_URL")),
                    formatGet(row, "RName"))
    Food_Query += ';\n'

file.write(Food_Query + '\n')
# print(Food_Query)

In [43]:
file.close()